In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
#%matplotlib inline
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
images = {
    "reference": {"filename":"../original_images/01.jpg"},
    "sterile_test": {"filename": "../original_images/02.jpg"},
    "real_world": {"filename": "../original_images/05.png"}
                    }
for image in images:
    images[image]["image"] = cv2.imread(images[image]["filename"])
    images[image]["image"] = cv2.cvtColor(images[image]["image"], cv2.COLOR_BGR2RGB)

In [3]:
fig, axes = plt.subplots(len(images))
fig.set_size_inches(6, 2.5*len(images))
for index, image in enumerate(["reference", "sterile_test", "real_world"]):
    axes[index].imshow(images[image]["image"])

In [4]:
# detectors are for finding keypoints.  They often also support computing
detectors = {"fast": cv2.FastFeatureDetector_create(), # lots of points (2159) all over
             "brisk": cv2.BRISK_create(),              
             "akaze": cv2.AKAZE_create(),  # strange delocalization
             "kaze": cv2.KAZE_create(),  # strange delocalization
             "agast": cv2.AgastFeatureDetector_create(),
             "gftt": cv2.GFTTDetector_create(),
             "mser": cv2.MSER_create(),  # very few keypoints (80)
             "orb": cv2.ORB_create(),
             "star": cv2.xfeatures2d.StarDetector_create(),
             "sift": cv2.xfeatures2d.SIFT_create(),
             "surf": cv2.xfeatures2d.SURF_create(),
            }
descriptors_only = {
    "freak": cv2.xfeatures2d.FREAK_create(),
    "latch": cv2.xfeatures2d.LATCH_create(),
    "lucid": cv2.xfeatures2d.LUCID_create(1, 1),
}

In [5]:
image = images["reference"]["image"]
for algorithm in detectors:
    plt.figure()
    kps = detectors[algorithm].detect(image)
    plt.imshow(cv2.drawKeypoints(image, kps, (255, 0,0)))
    plt.title(algorithm + " (%d pts)"%len(kps))

Compute features
================

In [11]:
def compute_features(image, detector_alg, descriptor_alg=None):
    data = image["image"]
    if descriptor_alg in detectors:
        kps, descriptors = detectors[descriptor_alg].detectAndCompute(data, None)
    elif descriptor_alg in descriptors_only:
        kps = detectors[detector_alg].detect(data)
        kps, descriptors = descriptors_only[descriptor_alg].compute(data, kps)
    else:
        raise ValueError("unknown algorithm passed to descriptor stage")
    image["kps"] = kps
    image["descriptors"] = descriptors
    return image

Match keypoint descriptors
==========================

In [12]:
def perspective_match(reference, unknown, use_flann=False, min_match_count=10):
    if use_flann:
        FLANN_INDEX_KDTREE = 0
        FLANN_INDEX_LSH    = 6
        # floating point algorithms
        if descriptor in ["sift", "surf"]:
            index_params = dict(algorithm = FLANN_INDEX_KDTREE,
                                trees = 5)
        # binary algorithms
        else:
            index_params= dict(algorithm = FLANN_INDEX_LSH,
                                table_number = 6, # 12
                                key_size = 12,     # 20
                                multi_probe_level = 1) #2
        search_params = dict(checks = 50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
    else:
        matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(reference["descriptors"],
                               unknown["descriptors"],
                               k=2)
    good = []
    matchesMask=None
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    if len(good)>min_match_count:
        src_pts = np.float32([ reference["kps"][m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ unknown["kps"][m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        # this limits matches to being within the identified subimage
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()

    else:
        print "Not enough matches are found (%d/%d)" % (len(good), min_match_count)
        matchesMask, good = None, None
    return matchesMask, good

In [27]:
def draw_matches(reference_features, unknown_features, mask, good_pts):
    fig = plt.figure()
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = (255,0,0),
                       matchesMask = mask,
                       flags = 2)

    img3 = cv2.drawMatches(reference_features["image"],
                           reference_features["kps"],
                           unknown_features["image"],
                           unknown_features["kps"],
                           good_pts,None,**draw_params)
    plt.imshow(img3)
    return fig

In [34]:
def wrapper(reference, unknown, detector_alg, 
            descriptor_alg=None, use_flann=False,
            min_match_count=5):
    if not descriptor_alg:
        descriptor_alg = detector_alg
    reference_features = compute_features(reference, 
                                          detector_alg, 
                                          descriptor_alg)
    unknown_features = compute_features(unknown, 
                                        detector_alg, 
                                        descriptor_alg)
    matchesMask, good_pts = perspective_match(reference_features,
                                              unknown_features,
                                             use_flann=use_flann,
                                             min_match_count=min_match_count)
    fig = draw_matches(reference_features, unknown_features, 
                 matchesMask, good_pts)
    fig.gca().set_title("keypoints: {}, detector: {}, Matcher: {}".format(
        detector_alg, descriptor_alg, 
        "FLANN" if use_flann else "Brute Force"))
    

In [35]:
wrapper(images["reference"], images["sterile_test"], "kaze")

In [36]:
wrapper(images["reference"], images["real_world"], "kaze")